In [5]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [8]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
0,0,Hay River,60.82,-115.80,28.40,63,40,5.82,CA,2020-04-20 04:17:04,scattered clouds,0.0,0.0
1,1,Garden City,42.33,-83.33,37.99,55,1,5.82,US,2020-04-20 04:21:44,clear sky,0.0,0.0
2,2,Half Moon Bay,37.46,-122.43,57.00,71,90,4.70,US,2020-04-20 04:19:45,overcast clouds,0.0,0.0
3,3,Smithers,54.78,-127.17,60.80,52,5,8.05,CA,2020-04-20 04:21:44,clear sky,0.0,0.0
4,4,Rikitea,-23.12,-134.97,76.84,71,93,6.76,PF,2020-04-20 04:15:46,overcast clouds,0.0,0.0


In [10]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
# Prompt the customer for the maximum temperature preference. 
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Prompt the customer to answer if he or she would like it to be raining or not.
rain = str(input('Do you want it to be raining? (yes/no)\n'))
# Prompt the customer to answer if he or she would like it to be snowing or not.
snow = str(input('Do you want it to be snowing? (yes/no)\n'))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no)
yes
Do you want it to be snowing? (yes/no)
no


In [11]:
# Filter the dataset to find the cities that fit the criteria.
for index, row in city_data_df.iterrows():
    if rain == "no":
        rain_inches = city_data_df["Rain inches (last 3 hours)"] == 0
    else:
        rain_inches = city_data_df["Rain inches (last 3 hours)"] != 0
    if snow == "no":
        snow_inches = city_data_df["Snow inches (last 3 hours)"] == 0
    else:
        snow_inches = city_data_df["Snow inches (last 3 hours)"] != 0

In [13]:
# Filter the dataset to find the cities that fit the criteria.

preferred_cities_df = city_data_df.loc[(city_data_df["Rain inches (last 3 hours)"] != rain_inches) &
                                       (city_data_df["Snow inches (last 3 hours)"] != snow_inches) &
                                       (city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                       
preferred_cities_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
13,13,Mogadishu,2.04,45.34,82.51,74,99,11.43,SO,2020-04-20 04:16:59,light rain,2.15,0.0
28,28,Khorixas,-20.37,14.97,72.54,44,79,2.75,NaN,2020-04-20 04:21:50,light rain,0.24,0.0
32,32,Katsuura,35.13,140.30,60.26,85,100,17.40,JP,2020-04-20 04:16:10,light rain,2.93,0.0
41,41,Albany,42.60,-73.97,50.00,73,94,1.01,US,2020-04-20 04:16:16,light rain,1.98,0.0
57,57,Poum,-20.23,164.02,80.60,76,100,8.79,NC,2020-04-20 04:21:57,light rain,0.89,0.0


In [14]:
# Create new DataFrame.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Mogadishu,SO,82.51,light rain,2.04,45.34,
28,Khorixas,NaN,72.54,light rain,-20.37,14.97,
32,Katsuura,JP,60.26,light rain,35.13,140.30,
41,Albany,US,50.00,light rain,42.60,-73.97,
57,Poum,NC,80.60,light rain,-20.23,164.02,
58,Balabac,PH,87.67,light rain,7.99,117.06,
62,Kavieng,PG,84.33,light rain,-2.57,150.80,
66,Bluff,NZ,55.99,light rain,-46.60,168.33,
99,Faanui,PF,82.24,light rain,-16.48,-151.75,
100,Japura,ID,88.66,light rain,-0.32,102.35,


In [26]:
# Import the dependencies.
import requests
import gmaps

# Import API key
from config import g_key

In [16]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Mogadishu,SO,82.51,light rain,2.04,45.34,Hotel Juba
28,Khorixas,NaN,72.54,light rain,-20.37,14.97,Khorixas Lodge
32,Katsuura,JP,60.26,light rain,35.13,140.30,Katsuura Hotel Mikazuki
41,Albany,US,50.00,light rain,42.60,-73.97,
57,Poum,NC,80.60,light rain,-20.23,164.02,Gîte kajeon
58,Balabac,PH,87.67,light rain,7.99,117.06,Sing and Swing Lodge
62,Kavieng,PG,84.33,light rain,-2.57,150.80,Nusa Island Retreat
66,Bluff,NZ,55.99,light rain,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
99,Faanui,PF,82.24,light rain,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
100,Japura,ID,88.66,light rain,-0.32,102.35,KOST dan HOMESTAY RUMAH KITA JOSH


In [17]:
#Infobox
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [29]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")